# Job 1 <a name="job1"></a>

Deve generare un report contenente, per ciascuna azione:

- data prima quotazione
- data ultima quotazione
- variazione percentuale della quotazione (tra primo e ultimo prezzo di chiusura nel dataset)
- prezzo massimo 
- prezzo minimo

Il report deve essere ordinato per valori decrescenti del secondo punto (dalla data di quotazione più recente alla più vecchia).

## Spark <a name="spark1"></a>

```python
historical_stock_prices = loadDataFromCsv()

# Viene creato un nuovo RDD con chiave = ticker e valore = (prezzo di chiusura, data)

ticker_date = historical_stock_prices
 .map(x -> (ticker,(close_price, price_date)))

    
# Vengono poi creati due RDD per calcolare la data minima e massima della quotazione

first_quot_date = ticker_date.reduceByKey(min_date(a, b))
last_quot_date = ticker_date.reduceByKey(max_date(a, b))


# Viene fatto il join tra i due precedenti RDD e poi si mappa per avere (ticker) -> (primo prezzo di chiusura, prima data, ultimo prezzo di chiusura, ultima data). Infine si mappa, calcolando la variazione percentuale, per ottenere (ticker) -> (prima data, ultima data, variazione percentuale)

percent_variation = first_quot_date
    .join(last_quot_date)
    .map((ticker, ((first_close, first_date), (last_close, last_date))) -> (ticker, (first_close, first_date, last_close, last_date))
    .map((ticker, (first_close, first_date, last_close, last_date) -> (ticker, (first_date, last_date, perc_var =  ((last_close-first_close)/first_close)*100)))))
  
    
# Viene mappato l'RDD iniziale per ottenere, per ogni ticker, il prezzo minimo ed il prezzo massimo
    
min_price = historical_stock_prices
    .map(x -> (ticker, min_price))
    .reduceByKey(min(a,b))

max_price = historical_stock_prices
    .map(x -> (ticker, max_price))
    .reduceByKey(min(a,b))

    
# Vengono joinati insieme i precedenti risultati per ottenere l'RDD finale ordinato per data, nella forma (ticker) -> data prima quotazione, data ultima quotazione, variazione percentuale, valore minimo e valore massimo

results = percent_variation
    .join(min_price)
    .join(max_price)
    .map((ticker, ((first_date, last_date, perc_var), min_price, max_price)) -> (ticker, (first_date, last_date, percent_var, min_price, max_price))
    .sortByDate()

```

# Job 2 <a name="job2"></a>

Generare un report contenente, per ciascun settore e per ciascun anno del periodo 2009-2018: 

- variazione percentuale della quotazione del settore nell'anno (somma prezzi di chiusura di tutte le azioni del settore, considerando la prima e l'ultima data di ogni azione aggregate) 
- azione del settore con incremento percentuale maggiore nell'anno (col valore)
- azione del settore con maggior volume di transazioni nell'anno (con valore)
	ordinare per nome del settore
    
Il report deve essere ordinato per nome del
settore.

## Spark <a name="spark2"></a>

```python
historical_stock_prices = loadDataFromCsv()

historical_stocks = loadDataFromCsv()


# Filtra i dati per estrarre soltanto quelli relativi al periodo compreso tra 2009 e 2018
# Mappa per ottenere (ticker) -> (prezzo di chiusura, volume, data)

historical_stock_prices_filtered = historical_stock_prices
    .filter(2009 <= year <= 2018)
    .map(x -> (ticker, (close_price, volume, date)))


# Mappa il secondo dataset per ottenere ticker -> settore (gli unici dati che ci interessano)

hs = historical_stocks
    .map(x -> (ticker, sector))

    
# Viene effettuato il join tra i due dataset e viene crato un nuovo RDD mappato per avere (settore, year, ticker) -> (prezzo di chiusura, volume, data)

hsp_sector = historical_stock_prices_filtered.join(hs)
    .map((ticker, ((close_price, volume, date), sector)) -> ((sector, year, ticker),(close, volume, date)))
    

# Vengono create due RDD che per ogni settore, anno e ticker restituiscono il prezzo di chiusura alla prima e all'ultima data dell'anno per quel settore

first_quotation_close = hsp_sector
    .reduceByKey_minDate() 
    .map(((sector, year, ticker),(close, volume, date)) -> ((sector, year, ticker),close))

last_quotation_close = hsp_sector
    .reduceByKey_maxDate()
    .map(((sector, year, ticker),(close, volume, date)) -> ((sector, year, ticker),close))

    
# Viene effettuato il join tra i precedenti due RDD e una map per calcolare e aggiungere, per ogni settore, anno e ticker, la variazione percentuale in quell'anno per quel settore

ticker_percent_variation = first_quotation_close
     .join(last_quotation_close)
     .map(((sector, year, ticker),(first_close, last_close)) -> ((sector, year, ticker),(first_close, last_close, percent_var = ((last_close-first_close)/first_close)*100)))
             

# Viene effettuata prima una map per avere per ogni settore e anno il ticker e la variazione percentuale
# Viene poi effettuata una reduce by key per avere il ticker con la variazione percentuale massima per quell'anno e in quel settore con indicazione di tale variazione
 
ticker_max_percent_var = ticker_percent_variation
    .map(((sector, year, ticker),(first_close, last_close, percent_var )) -> ((sector, year), (ticker, percent_var)))
    .reduceByKey(max_value(a, b))

    
# Per ogni settore, anno e ticker calcola la somma dei volumi e poi prende il ticker con il massimo valore di somma di volumi per settore e anno

ticker_max_volume = hsp_sector
    .map(((sector, year, ticker),(close, volume, date)) -> ((sector, year, ticker), volume))
    .reduceByKey(volume_a + volume_b)
    .map(((sector, year, ticker), max_volume) -> ((sector, year), (ticker, max_volume)))
    .reduceByKey(max_value(a, b))


# Viene effettuata una map rimuove la variazione precentuale dal valore e il ticker dalla chiave
# Poi una seconda map per avere per ogni settore e anno la variazione percentuale totale

sector_year_percent_variation = ticker_percent_variation
    .map(((sector, year, ticker),(first_close, last_close, percent_var )) -> ((sector, year), (first_close, last_close)))
    .reduceByKey(sum_tuple(a, b))
    .map(((sector, year), (sum_first_close, sum_last_close)) -> ((sector, year), (total_percent_variation = (sum_last_close - sum_first_close)/sum_first_close)*100))


# Aggrega tutti i risultati intermedi e calcola i record finali per ottenere, per ogni settore e anno, la massima variazione della quotazione del settore nell'anno, l'azione con incremento maggiore nel settore e l'azione del settore con il maggior volume, con indicazione di tali valori. 

results = sector_year_percent_variation
    .join(ticker_max_percent_var)
    .map(((sector, year), (total_percent_variation, (max_ticker, max_percent_var)) -> ((sector, year), (total_percent_variation, max_ticker, max_percent_var))))
    .join(ticker_max_volume)
    .map(((sector, year), ((total_percent_variation, max_ticker, max_percent_var), (ticker, max_volume))) -> ((sector, year), (total_percent_variation, max_ticker, max_percent_var, ticker, max_volume)))     
    .sortBySectorYear()

```

# Job 3 <a name="job3"></a>

Generare le coppie di aziende che si somigliano (sulla base di una soglia = 1%) in termini di
variazione percentuale mensile nell’anno 2017. 

Mostrare l’andamento mensile delle due aziende nel formato:

- 1:{Apple, Intel}:   
    - GEN: Apple +2%, Intel +2,5%, 
    - FEB: Apple +3%, Intel +2,7%, 
    - MAR: Apple +0,5%, Intel +1,2%, ...
- 2:{Amazon, IBM}:    
    - GEN: Amazon +1%, IBM +0,5%, 
    - FEB: Amazon +0,7%, IBM +0,5%, 
    - MAR: Amazon +1,4%, IBM +0,7%, ...


## Spark <a name="spark3"></a>

```python
historical_stock_prices = loadDataFromCsv()


# Filtra i dati per ottenere soltanto quelli relativi all'anno 2017

ticker_close_date = historical_stock_prices
    .filter(x -> year(price_date) == 2017)
    .map(x -> (ticker, close_price, price_date))

    
# Mappa i valori per avere, per ogni ticker e mese, la tupla originale corrispondente

ticker_month_to_list = ticker_close_date
    .map((ticker, close_price, price_date) -> ((ticker, month(price_date), (ticker, close_price, price_date))))
        

# Reduce By Key per ottenere, per ogni ticker e mese, la prima data e il primo prezzo disponibili in quel mese per quel ticker
# Map per ottenere, per ogni ticker e mese, il prezzo minimo

ticker_month_to_mindate = ticker_month_to_list
    .reduceByKey(min_price_and_date)
    .map(((ticker, month), (ticker, min_close_price, min_price_date)) -> ((ticker, month), min_close_price))
        

# Reduce By Key per ottenere, per ogni ticker e mese, l'ultima data e l'ultimo prezzo disponibili in quel mese per quel ticker
# Map per ottenere, per ogni ticker e mese, il prezzo massimo

ticker_month_to_maxdate = ticker_month_to_list
    .reduceByKey(max_price_and_date)
    .map(((ticker, month), (ticker, max_close_price, max_price_date)) -> ((ticker, month), max_close_price))


# Viene effettuato il join tra i due RDD precedenti e una map per ottere, per ogni ticker e mese, la variazione percentuale per quel ticker in quel mese

ticker_month_variation = ticker_month_to_mindate
    .join(ticker_month_to_maxdate)
    .map(((ticker, month), (min_close_price, max_close_price)) -> ((ticker, month), perc_variation = ((max_close_price - min_close_price)/min_close_price)*100))
        

# Raggruppa le variazioni percentuali di tutti i mesi per ogni ticker 
# Filtra i record non relativi ad un intero anno
# Ordina per mese
ticker_aggregate_months = ticker_month_variation
    .map(((ticker, month), perc_variation) -> (ticker, (month, variation)))
    .groupByKey()
    .filter(length(list of (month, variation)) == 12)
    .map((ticker, list of (month, variation)) -> (ticker, sorted list by month))


# Effettua il prodotto cartesiano per individuare tutte le possibili coppie di ticker
# Filtra i record univoci e quelli che si somigliano in base ad una soglia (1%) in termini di variazione percentuale mensile

ticker_pairs_threshold = ticker_aggregate_months
    .cartesian(ticker_aggregate_months)
    .filter(ticker_1 < ticker_2 and abs(variation_1 - variation_2) < 1)
    .map(
        ((ticker_1, sorted list by month),(ticker_2, sorted list by month)) -> ((ticker_1, ticker_2), merged list of months))
     
```